In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import keras 
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.regularizers import l2
from keras import callbacks

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2560548733057308660
 xla_global_id: -1]

In [ ]:
output_classes = 2
image_height, image_width = 580, 400
image_shape = (image_height, image_width)
epochs = 10;


In [ ]:
train_data = r'/content/drive/MyDrive/Kamienie01/Dane/filtered/train_set'
val_data=r'/content/drive/MyDrive/Kamienie01/Dane/filtered/validation_set'
test_data = r'/content/drive/MyDrive/Kamienie01/Dane/filtered/test_set'

In [ ]:
#if k.image_data_format() == 'channels_first':
#  input_shape = (3, img_width, img_height)
#else:
#  input_shape = (img_width, img_height, 3)

In [ ]:
print('train: ')
for root,dirs,files in os.walk(train_data):
    print (root, len(files))
print('val: ')
for root,dirs,files in os.walk(val_data):
    print (root, len(files))
print('test: ')
for root,dirs,files in os.walk(test_data):
    print (root, len(files))

train: 
/content/drive/MyDrive/Kamienie01/Dane/filtered/train_set 0
/content/drive/MyDrive/Kamienie01/Dane/filtered/train_set/t 1859
/content/drive/MyDrive/Kamienie01/Dane/filtered/train_set/NONE 3718
val: 
/content/drive/MyDrive/Kamienie01/Dane/filtered/validation_set 0
/content/drive/MyDrive/Kamienie01/Dane/filtered/validation_set/NONE 2222
/content/drive/MyDrive/Kamienie01/Dane/filtered/validation_set/t 1111
test: 
/content/drive/MyDrive/Kamienie01/Dane/filtered/test_set 0
/content/drive/MyDrive/Kamienie01/Dane/filtered/test_set/NONE 13379
/content/drive/MyDrive/Kamienie01/Dane/filtered/test_set/t 1668


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip = True,
)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)

val_generator = train_datagen.flow_from_directory(
    val_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size = (image_shape),
    batch_size = 32,
    class_mode = 'binary'
)


Found 5577 images belonging to 2 classes.
Found 3333 images belonging to 2 classes.
Found 15047 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.DenseNet121(
    weights='imagenet', 
    include_top=False, 
    input_tensor=Input(shape=(image_shape+(3,)))
    )
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

model = Model(inputs=base_model.input,outputs=prediction)

29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
'''
model.compile(optimizer= 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])
'''
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
#przerywamy jeśli wystepuje overfitting
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

train_steps = 5577//32
val_steps = 5577//32


history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs = 32,
    validation_data=val_generator,
    validation_steps=12
)

Epoch 1/32
